Run the code following two steps:


1.   List the features returned by the image analysis tool (Rekognition/Vision/Azure) by excuting one of the three code blocks
2. Run the final code block to evaluate the framework.  



In [ ]:
!unzip Data.zip

Archive:  Data.zip
   creating: Data/FacialData/
   creating: Data/FacialData/Amazon/
  inflating: Data/FacialData/Amazon/user_31.csv  
   creating: Data/FacialData/Google/
  inflating: Data/FacialData/Google/user_31.csv  
   creating: Data/FacialData/Microsoft/
  inflating: Data/FacialData/Microsoft/user_31.csv  
   creating: Data/Self_reports/
   creating: Data/Self_reports/Arousal/
  inflating: Data/Self_reports/Arousal/user_31.csv  
   creating: Data/Self_reports/Valence/
  inflating: Data/Self_reports/Valence/user_31.csv  


In [ ]:
!pip install pyDML

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyDML: filename=pyDML-0.1.0-cp310-cp310-linux_x86_64.whl size=5883453 sha256=2b8c1c8f94d1287127bb8341135137c3339d065804d70cf91e09aadd3836d331
  Stored in directory: /root/.cache/pip/wheels/dc/e3/c4/178953a1ef489ed48dcd682135ce3b39b4c6e8b13db6739216
Successfully built pyDML


In [ ]:
import pandas as pd
import numpy as np
from math import *
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from keras.utils import np_utils
from keras import optimizers
import keras.backend as K
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.decomposition import KernelPCA
from sklearn.metrics import classification_report
import os
import csv
import matplotlib.pyplot as plt
import fnmatch
import datetime
import shutil
from statistics import mean
from sklearn.feature_selection import f_classif
from itertools import combinations
import math
from numpy import percentile
from dml import kda

Necessary functions to compute self-report features

In [ ]:
def find_transition_matrix(values):
	src_labels=[0,1]
	tgt_labels=[0,1]
	
	transition_vector=[]
	
	for em_src in src_labels:
		src=values[values[:,0]==em_src,:]
		if src.shape[0] != 0:
			for em_tgt in tgt_labels:
				tgt=src[src[:,1]==em_tgt,:]
				trans_prob=tgt.shape[0]/float(src.shape[0])			
				transition_vector.append(trans_prob)
		else:
			transition_vector.extend([0,0])
	return transition_vector

def find_average_sequence_length(values):
	seq_length_vector=[]

	seq_lengths_l=np.empty((0))
	seq_lengths_h=np.empty((0))


	count=1
	for i in range(1,values.shape[0]):
		if values[i,1]==0 and values[i,1]==values[i-1,1]:				
				count=count+1
				continue
		else:
			if values[i-1,1]==0:
				seq_lengths_l=np.append(seq_lengths_l,count)
				count=1
	if count>1 or seq_lengths_l.shape[0]==0:
		seq_lengths_l=np.append(seq_lengths_l,count)	

	count=1
	for i in range(1,values.shape[0]):
		if values[i,1]==values[i-1,1] and values[i,1]==1:
			count=count+1
			continue
		else:
			if values[i-1,1]==1:			
				seq_lengths_h=np.append(seq_lengths_h,count)
				count=1
	if count>1 or seq_lengths_h.shape[0]==0:
		seq_lengths_h=np.append(seq_lengths_h,count)

		

	seq_length_vector.append(int(round(np.percentile(seq_lengths_l,75))))
	seq_length_vector.append(int(round(np.percentile(seq_lengths_h,75))))
	

	#print('seq vector:',np.array(seq_length_vector))
	return np.array(seq_length_vector)

def find_PRE(transition_vector,previous_label):
	if previous_label==0:
		label_vector=np.array([1,0])
	elif previous_label==1:
		label_vector=np.array([0,1])

	
	transition_matrix=np.empty((0,2))
	transition_matrix=np.append(transition_matrix,np.array([transition_vector[0:2]]),axis=0)
	transition_matrix=np.append(transition_matrix,np.array([transition_vector[2:4]]),axis=0)


	predicted_val=np.matmul(label_vector,transition_matrix)
	idx=np.argmax(predicted_val)

	predicted_val_sorted=np.sort(predicted_val)

	r=np.where(predicted_val==predicted_val_sorted[0])
	sec_idx=r[0][0]

	
	
	if idx==0:
		pre='low'
	elif idx==1:
		pre='high'


	if sec_idx==0:
		pre_2='low'
	elif sec_idx==1:
		pre_2='high'
	

	total_wt=np.sum(predicted_val)
	if total_wt==0:
		predicted_val_norm=np.array([0,0])
	else:
		predicted_val_norm=np.divide(predicted_val,total_wt)

	return pre,pre_2,predicted_val_norm

def find_val(label):
   if label=='low':
     val=0
   elif label=='high':
     val=1
   return val  
def find_label(val):
	if val==0:
		label='low'
	elif val==1:
		label='high'

	return label

Return facial features value

In [ ]:
def facial_feature(photo,common):
  user_no=photo.split('/')[0]
  # mention facial feature file
  # for Amazon
  user_file='Data/FacialData/Amazon/'+user_no+'.csv'
  # for vision
  #user_file='Data/FacialData/Google/'+user_no+'.csv'
  # for Azure
  # user_file='Data/FacialData/Microsoft/'+user_no+'.csv'
  user_data=pd.read_csv(user_file)
  df=user_data[user_data['FileName']==photo]
  df=df[common]
  #print(df)
  feature_list=[]

  df=df.to_numpy()
  #print(df)
  for k in range(df.shape[1]):
    feature_list.append(df[0,k])
  #print(feature_list)
  return feature_list

Remove duplicate data samples if any

In [ ]:
def data_stat(train_data):
  np.set_printoptions(suppress=True)
  uniquerows=np.unique(train_data,axis=0)
  print("number of unique rows=",uniquerows.shape)
  return uniquerows

Build the model and evaluate

In [ ]:
def Random_Forest_model(train_data,test_data):

  
  print("train size in model function=",train_data.shape[0],"test size in model function=",test_data.shape[0])
  train_data=data_stat(train_data)
  print("train size after removing duplicates=",train_data.shape[0],"test size after removing duplicates=",test_data.shape[0])
  
  X_train=train_data[:,1:train_data.shape[1]]
  Y_train=train_data[:,0]
  X_test=test_data[:,1:test_data.shape[1]]
  Y_test=test_data[:,0]
  
  

  # define model
  model = RandomForestClassifier(n_estimators=50, random_state=42)
  
  s=model.fit(X_train,Y_train)

  Y_pred=s.predict(X_test)
  Y_predTrain=s.predict(X_train)
  #importance = s.feature_importances_
  #print("Y_train=",Y_train)
  
  print("Y_test=",Y_test)
  print("Y_pred=",Y_pred)
  
  
  # f1 score
  print("Train accuracy=",f1_score(Y_train,Y_predTrain,average="macro"))
  train_score=f1_score(Y_train,Y_predTrain,average="macro")
  print("Test f1=",f1_score(Y_test,Y_pred,average="macro"))
  test_score=f1_score(Y_test,Y_pred,average="macro")
  print("Test accuracy=",accuracy_score(Y_test,Y_pred)) 
  accu_score=accuracy_score(Y_test,Y_pred)
  print(classification_report(Y_test, Y_pred))
  

  return train_score,test_score,Y_pred


Create train and test set

In [ ]:
def feature_file_creation(train_data,test_data,common):
  # column_list need to add as argument when using correlation
  feature_no=len(common)
  accu_score=0
  et_norm=np.empty((0))
  et_from_last_label=np.array([0,0])
  train_feature=np.empty((0,feature_no+4))
  test_feature=np.empty((0,feature_no+4))
  transition_matrix=find_transition_matrix(train_data)
  
  continuous_seq_len=1
  
  for i in range(0,train_data.shape[0]):
  
    pre,pre_2,label_wt=find_PRE(transition_matrix,train_data[i,0])
    # compute elapsed time
    if train_data[i,0]==0:
          et_from_last_label[0]=train_data[i,4]
          et_from_last_label[1]=et_from_last_label[1]+train_data[i,4]
    elif train_data[i,0]==1: 
          et_from_last_label[1]=train_data[i,4] 
          et_from_last_label[0]=et_from_last_label[0]+train_data[i,4]
        
    max_et=np.amax(et_from_last_label)
    
    if(max_et==0):
           continue
    et_norm=np.divide(et_from_last_label,float(max_et))
    # influence calculation
    for j in range(0,2):
          et_norm[j]=1-et_norm[j]
          label_wt[j]=label_wt[j]*et_norm[j]
    #print("influence:",label_wt)

    # sequence length calculation
    if(i==0):
       continuous_seq_len=1
    else:

      if train_data[i,0]==train_data[i,1]:
          continuous_seq_len=continuous_seq_len+1
      else:
          continuous_seq_len=1

    img_features=facial_feature(train_data[i,3],common)  
    self_report=train_data[i,1]
    train_instance_lst=[]
    train_instance_lst.append(self_report)
    train_instance_lst.append(label_wt[0])
    train_instance_lst.append(label_wt[1])
    train_instance_lst.append(continuous_seq_len)
    train_instance_lst.extend(img_features)
    
    
    train_instance=np.array(train_instance_lst).reshape(1,len(train_instance_lst))
    
    # training data
    train_feature=np.append(train_feature,train_instance,axis=0)
    
  
  # for test feature
  
  for p in range(0,test_data.shape[0]):
    old_label=test_data[p,0]
    pre,pre_2,label_wt=find_PRE(transition_matrix,old_label) 
    if old_label==0:
          et_from_last_label[0]=test_data[p,4]
          et_from_last_label[1]=et_from_last_label[1]+test_data[p,4]
    elif old_label==1:
          et_from_last_label[1]=test_data[p,4]
          et_from_last_label[0]=et_from_last_label[0]+test_data[p,4]
        
    max_et=np.amax(et_from_last_label)
    
    if(max_et==0):

          continue
    et_norm=np.divide(et_from_last_label,float(max_et))
    for j in range(0,2):
          et_norm[j]=1-et_norm[j]
          label_wt[j]=label_wt[j]*et_norm[j]

    # sequence length calculation      
    
    if(test_data[p,0]==find_val(pre)):
             continuous_seq_len+=1
    else:    
          continuous_seq_len=1
    

    img_features=facial_feature(test_data[p,3],common)      
    self_report=test_data[p,1] 
    test_instance_lst=[]
    # typing feature considered
    test_instance_lst.append(self_report)
    test_instance_lst.append(label_wt[0])
    test_instance_lst.append(label_wt[1])
    test_instance_lst.append(continuous_seq_len)
    test_instance_lst.extend(img_features)
    
    test_instance=np.array(test_instance_lst).reshape(1,len(test_instance_lst))

    test_feature=np.append(test_feature,test_instance,axis=0)
    
    

 
  return train_feature,test_feature

Excute the below code block to list the features obtain from Amazon Rekognition

In [ ]:
# driver code
user_no=15
common=[]
user_name='user_'+str(format(user_no,'02'))

dirname='Data/FacialData/Amazon/'+user_name+'.csv'

array=pd.read_csv(dirname)
array=array.drop_duplicates(keep='last')
array=array.drop(['FileName',"TapEmotion",'NewTarget'],axis=1)
array=array.drop(['SmileValue_ False','SmileValue_ True','EyeglassesValue_ False','SunglassesValue_ False','SunglassesValue_ True',"GenderValue_ 'Female'","GenderValue_ 'Male'","BeardValue_ False",
                            "BeardValue_ True","MustacheValue_ False","EyesOpenValue_ False","EyesOpenValue_ True","MouthOpenValue_ False","MouthOpenValue_ True","MustacheValue_ True",'EyeglassesValue_ True'],axis=1,errors='ignore')  
array=array.drop(['Confidence','BoundingBoxWidth','BoundingBoxHeight','BoundingBoxLeft','BoundingBoxTop','AgeRangeLow','AgeRangeHigh','SmileConfidence', 'EyesOpenConfidence', 'MouthOpenConfidence','EyeglassesConfidence','SunglassesConfidence','GenderConfidence','BeardConfidence','MustacheConfidence','Date'], axis=1,errors='ignore')                          
column_list=list(array.columns)
print(array.shape)
common=column_list
print(common)

(510, 65)
['eyeLeftX', 'eyeLeftY', 'eyeRightX', 'eyeRightY', 'mouthLeftX', 'mouthLeftY', 'mouthRightX', 'mouthRightY', 'noseX', 'noseY', 'leftEyeBrowLeftX', 'leftEyeBrowLeftY', 'leftEyeBrowRightX', 'leftEyeBrowRightY', 'leftEyeBrowUpX', 'leftEyeBrowUpY', 'rightEyeBrowLeftX', 'rightEyeBrowLeftY', 'rightEyeBrowRightX', 'rightEyeBrowRightY', 'rightEyeBrowupX', 'rightEyeBrowupY', 'leftEyeLeftX', 'leftEyeLeftY', 'leftEyeRightX', 'leftEyeRightY', 'leftEyeUpX', 'leftEyeUpY', 'leftEyeDownX', 'leftEyeDownY', 'rightEyeLeftX', 'rightEyeLeftY', 'rightEyeRightX', 'rightEyeRightY', 'rightEyeUpX', 'rightEyeUpY', 'rightEyeDownX', 'rightEyeDownY', 'noseLeftX', 'noseLeftY', 'noseRightX', 'noseRightY', 'mouthUpX', 'mouthUpY', 'mouthDownX', 'mouthDownY', 'leftPupilX', 'leftPupilY', 'rightPupilX', 'rightPupilY', 'upperJawlineLeftX', 'upperJawlineLeftY', 'midJawlineLeftX', 'midJawlineLeftY', 'chinBottomX', 'chinBottomY', 'midJawlineRightX', 'midJawlineRightY', 'upperJawlineRightX', 'upperJawlineRightY', 'Po

Excute the code to list the features obtain from Google Vision

In [ ]:
# # for vision data processing
# # driver code
# user_no=31
# common=[]
# user_name='user_'+str(format(user_no,'02'))
# dirname='Data/FacialData/Google/'+user_name+'.csv'

# array=pd.read_csv(dirname)
# array=array.drop_duplicates(keep='last')
# array=array.drop(['FileName'],axis=1)

# array=array.drop(['bounding_poly1x', 'bounding_poly1y', 'bounding_poly2x', 'bounding_poly2y', 'bounding_poly3x', 'bounding_poly3y', 'bounding_poly4x', 'bounding_poly4y', 'fdbounding_poly1x', 'fdbounding_poly1y', 'fdbounding_poly2x', 'fdbounding_poly2y', 'fdbounding_poly3x', 'fdbounding_poly3y', 'fdbounding_poly4x', 'fdbounding_poly4y','detection_confidence','landmarking_confidence','joy_likelihood','sorrow_likelihood','anger_likelihood','surprise_likelihood','emotion','target','Date'],axis=1,errors='ignore')  
             
# column_list=list(array.columns)
# #print(column_list)
# print(array.shape)
# common=column_list
# print(common)

Excute the code to list features obtain from Microsoft Azure

In [ ]:
# # for Azure Data Processing
# user_no=12
# common=[]
# user_name='user_'+str(format(user_no,'02'))
# dirname='Data/FacialData/Microsoft/'+user_name+'.csv'
# array=pd.read_csv(dirname)
# array=array.drop_duplicates(keep='last')
# array=array.drop(['FileName'],axis=1)
# column_list=list(array.columns)
# print(array.shape)
# common=column_list
# print(common)

(719, 40)
['pupil_leftX', 'pupil_leftY', 'pupil_rightX', 'pupil_rightY', 'nose_tipX', 'nose_tipY', 'mouth_leftX', 'mouth_leftY', 'mouth_rightX', 'mouth_rightY', 'eyebrow_left_outerX', 'eyebrow_left_outerY', 'eyebrow_left_innerX', 'eyebrow_left_innerY', 'eye_left_outerX', 'eye_left_outerY', 'eye_left_topX', 'eye_left_topY', 'eye_left_bottomX', 'eye_left_bottomY', 'eye_left_innerX', 'eye_left_innerY', 'eyebrow_right_innerX', 'eyebrow_right_innerY', 'eyebrow_right_outerX', 'eyebrow_right_outerY', 'eye_right_innerX', 'eye_right_innerY', 'eye_right_topX', 'eye_right_topY', 'nose_right_alar_topX', 'nose_right_alar_topY', 'nose_left_alar_out_tipX', 'nose_left_alar_out_tipY', 'nose_right_alar_out_tipX', 'nose_right_alar_out_tipY', 'upper_lip_topX', 'upper_lip_topY', 'upper_lip_bottomX', 'upper_lip_bottomY']


In [ ]:
# driver code
import pickle
dic={}
for user_no in range(11,34):
  common=[]
  user_name='user_'+str(format(user_no,'02'))

  dirname='Data/FacialData/Amazon/'+user_name+'.csv'

  array=pd.read_csv(dirname)
  array=array.drop_duplicates(keep='last')
  array=array.drop(['FileName',"TapEmotion",'NewTarget'],axis=1)
  array=array.drop(['SmileValue_ False','SmileValue_ True','EyeglassesValue_ False','SunglassesValue_ False','SunglassesValue_ True',"GenderValue_ 'Female'","GenderValue_ 'Male'","BeardValue_ False",
                              "BeardValue_ True","MustacheValue_ False","EyesOpenValue_ False","EyesOpenValue_ True","MouthOpenValue_ False","MouthOpenValue_ True","MustacheValue_ True",'EyeglassesValue_ True'],axis=1,errors='ignore')  
  array=array.drop(['Confidence','BoundingBoxWidth','BoundingBoxHeight','BoundingBoxLeft','BoundingBoxTop','AgeRangeLow','AgeRangeHigh','SmileConfidence', 'EyesOpenConfidence', 'MouthOpenConfidence','EyeglassesConfidence','SunglassesConfidence','GenderConfidence','BeardConfidence','MustacheConfidence','Date'], axis=1,errors='ignore')                          
  column_list=list(array.columns)
  common=column_list

  dirname='Data/Self_reports/Valence/'

  for user in [user_no]:  
    # read user file
    test_user='user_'+str(format(user,'02'))
    test_dir=dirname+test_user+'.csv'
    if(os.path.isfile(test_dir)):

      dataset=pd.read_csv(test_dir)
      dataset=dataset.drop_duplicates(keep='last')
    else:

      continue  

    test_data=dataset.values
    data_size=test_data.shape[0]  
    print("data size=",data_size)
    # specify training and testing data size for each iteration of cross-validation
    test_data_size=ceil(data_size*0.25)
    train_data_size=ceil(data_size*0.75)
    start=0
    train_data_index=0
    test_data_index=0

    accu_list=[]
    
    train_accu=[] 
    i=1
    

  
    array1=np.array([])

    print("fold no=",i)
    train_data_index=start+train_data_size
    test_data_index=start+train_data_size+test_data_size
    print("train data index=",start,"-->",train_data_index-1,"test data index=",train_data_index,"--->",test_data_index-1)
    print("train_data size=",test_data[start:train_data_index,:].shape[0],"test data size=",test_data[train_data_index:test_data_index,:].shape[0])
    train_dataset=test_data[start:train_data_index,:]
    test_dataset=test_data[train_data_index:test_data_index,:]
    
    
    # calculate number of high and low samples in the train set
    if(test_dataset.shape[0]!=0):
      high_train=0
      low_train=0
      for k in range(train_dataset.shape[0]):
            if(train_dataset[k][1]==0):
              low_train+=1
            elif(train_dataset[k][1]==1):
              high_train+=1
      print("no. of high samples in train=",high_train,"no of low samples in train=",low_train) 
      if(low_train==0 or high_train==0):
        continue
      train_feature,test_feature=feature_file_creation(train_dataset,test_dataset,common)
      array1=train_dataset
      
      
      print("train feature shape:",train_feature.shape)
      print("test feature shape:",test_feature.shape)
      # separte targets from features
      X_train=train_feature[:,1:train_feature.shape[1]]   
      
      Y_train=train_feature[:,0]
      X_test=test_feature[:,1:test_feature.shape[1]]
    
      Y_test=test_feature[:,0]
      # seperate facial features
      new_train=X_train[:,3:X_train.shape[1]]
      new_test=X_test[:,3:X_test.shape[1]]
      
      # do standarization on facial data
      
      scaler = StandardScaler()
      #scaler.fit(X_train)
      scaler.fit(new_train)
      new_train=scaler.transform(new_train)
      new_test=scaler.transform(new_test)
      np.set_printoptions(suppress=True)
      

      
      
      # reduce dimension of features
      
      # apply PCA 
      pca=KernelPCA(n_components=1,kernel='rbf',eigen_solver='arpack',remove_zero_eig=True,random_state=32)
      
      pca.fit(new_train)
      new_train=pca.transform(new_train)
      new_test=pca.transform(new_test)
      
      
      

      # apply KLDA
      # uncomment the below code when KLDA is applied
      '''
      klda=kda.KDA(n_components=1,kernel='rbf')
      new_train=klda.fit_transform(new_train,Y_train)
      new_test=klda.transform(new_test)
      '''
      
      # concatenate self-report features and facial features (obtain from feature reduction tool)
      trainX_new=np.concatenate((X_train[:,0:3],new_train),axis=1)
      testX_new=np.concatenate((X_test[:,0:3],new_test),axis=1)
      
      print(trainX_new.shape)
      print(testX_new.shape)
      Y_train=Y_train.reshape(len(Y_train),1)
      Y_test=Y_test.reshape(len(Y_test),1)
      
      
      final_train=np.concatenate((Y_train,trainX_new),axis=1)
      final_test=np.concatenate((Y_test,testX_new),axis=1)
      
      
      train_score,score,Y_pred=Random_Forest_model(final_train,final_test)  
      ################# our info
      dic[user_no]=(train_data_index,test_data_index-1,Y_pred)
      #################
      accu_list.append(score)
      
      train_accu.append(train_score)

      start=start+test_data_size

print(dic)        
pickle.dump(dic,open("selfi/Arousal/prediction_valence","wb"))
    # # print the accuracy     
    # print("f1 score list=",accu_list)
    # if(len(train_accu)!=0):   
    #     print("Train f1-score=",mean(train_accu))    
    # if(len(accu_list)!=0):   
    #     print("Test f1-score=",mean(accu_list))

data size= 649
fold no= 1
train data index= 0 --> 486 test data index= 487 ---> 649
train_data size= 487 test data size= 162
no. of high samples in train= 295 no of low samples in train= 192
train feature shape: (486, 69)
test feature shape: (162, 69)
(486, 4)
(162, 4)
train size in model function= 486 test size in model function= 162
number of unique rows= (486, 5)
train size after removing duplicates= 486 test size after removing duplicates= 162
Y_test= [1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0.
 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1.]
Y_pred= [0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


data size= 149
fold no= 1
train data index= 0 --> 111 test data index= 112 ---> 149
train_data size= 112 test data size= 37
no. of high samples in train= 39 no of low samples in train= 73
train feature shape: (112, 69)
test feature shape: (37, 69)
(112, 4)
(37, 4)
train size in model function= 112 test size in model function= 37
number of unique rows= (112, 5)
train size after removing duplicates= 112 test size after removing duplicates= 37
Y_test= [0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0.]
Y_pred= [1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0.]
Train accuracy= 1.0
Test f1= 0.456497175141243
Test accuracy= 0.6486486486486487
              precision    recall  f1-score   support

         0.0       0.79      0.77      0.78        30
         1.0       0.12      0.14      0.13         7

    accuracy                           0.65        37
   macro avg

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train feature shape: (144, 69)
test feature shape: (47, 69)
(144, 4)
(47, 4)
train size in model function= 144 test size in model function= 47
number of unique rows= (144, 5)
train size after removing duplicates= 144 test size after removing duplicates= 47
Y_test= [1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0.]
Y_pred= [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Train accuracy= 1.0
Test f1= 0.47275641025641024
Test accuracy= 0.7021276595744681
              precision    recall  f1-score   support

         0.0       0.71      0.97      0.82        33
         1.0       0.50      0.07      0.12        14

    accuracy                           0.70        47
   macro avg       0.61      0.52      0.47        47
weighted avg       0.65      0.70      0.61        47

data size= 121
fold no= 1
tra

...

data size= 99
fold no= 1
train data index= 0 --> 74 test data index= 75 ---> 99
train_data size= 75 test data size= 24
no. of high samples in train= 73 no of low samples in train= 2
train feature shape: (74, 69)
test feature shape: (24, 69)
(74, 4)
(24, 4)
train size in model function= 74 test size in model function= 24
number of unique rows= (74, 5)
train size after removing duplicates= 74 test size after removing duplicates= 24
Y_test= [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Y_pred= [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Train accuracy= 1.0
Test f1= 1.0
Test accuracy= 1.0
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00        24

    accuracy                           1.00        24
   macro avg       1.00      1.00      1.00        24
weighted avg       1.00      1.00      1.00        24

data size= 130
fold no= 1
train data index= 0 --> 97 test data index= 98 --->

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


data size= 36
fold no= 1
train data index= 0 --> 26 test data index= 27 ---> 35
train_data size= 27 test data size= 9
no. of high samples in train= 15 no of low samples in train= 12
train feature shape: (27, 69)
test feature shape: (9, 69)
(27, 4)
(9, 4)
train size in model function= 27 test size in model function= 9
number of unique rows= (27, 5)
train size after removing duplicates= 27 test size after removing duplicates= 9
Y_test= [0. 0. 1. 0. 1. 1. 0. 0. 0.]
Y_pred= [0. 1. 0. 1. 1. 1. 1. 1. 1.]
Train accuracy= 1.0
Test f1= 0.325
Test accuracy= 0.3333333333333333
              precision    recall  f1-score   support

         0.0       0.50      0.17      0.25         6
         1.0       0.29      0.67      0.40         3

    accuracy                           0.33         9
   macro avg       0.39      0.42      0.33         9
weighted avg       0.43      0.33      0.30         9

data size= 45
fold no= 1
train data index= 0 --> 33 test data index= 34 ---> 45
train_data size= 34 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train feature shape: (95, 69)
test feature shape: (31, 69)
(95, 4)
(31, 4)
train size in model function= 95 test size in model function= 31
number of unique rows= (95, 5)
train size after removing duplicates= 95 test size after removing duplicates= 31
Y_test= [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 1.]
Y_pred= [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 1. 1. 0.]
Train accuracy= 1.0
Test f1= 0.5694444444444444
Test accuracy= 0.8064516129032258
              precision    recall  f1-score   support

         0.0       0.33      0.20      0.25         5
         1.0       0.86      0.92      0.89        26

    accuracy                           0.81        31
   macro avg       0.60      0.56      0.57        31
weighted avg       0.77      0.81      0.79        31

{11: (487, 649, array([0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 0., 0., 1., 1